<a href="https://colab.research.google.com/github/cappelchi/calcio_notebooks/blob/main/draft/football_GRAPH_prediction_disjoint_spektral_models_train_test_221029.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Project config

In [ ]:
!pip install neptune-client neptune-tensorflow-keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 254 kB 8.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 829 kB 62.7 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 182 kB 42.9 MB/s 
     |████████████████████████████████| 132 kB 42.8 MB/s 
     |████████████████████████████████| 9.3 MB 52.8 MB/s 
     |████████████████████████████████| 79 kB 10.4 MB/s 
     |████████████████████████████████| 140 kB 64.5 MB/s 
     |████████████████████████████████| 67 kB 6.3 MB/s 
     |████████████████████████████████| 130 kB 62.8 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 127 kB 55.8 MB/s 
     |████████████████████████████████| 93 kB 2.1 MB/s 
     |█████████████████████████

In [ ]:
import neptune.new as neptune
#from neptune.new.integrations.tensorflow_keras import NeptuneCallback
def get_credential(frmwork = 'neptune_team'):
    with open('cred_andrey.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw

In [ ]:
#@title Set API key for neptune.ai
set_api = True #@param {type:"boolean"}
if set_api:
    username, api_key = get_credential()

### Installations

In [ ]:
!pip install spektral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 6.9 MB/s 


### Downloads

In [ ]:
data_version = 'data_graph_221019/'
validation_dataset_name = './prem_validation.csv'
dataset_version = 'data/dataset_val_prod_0818'
project = neptune.init_project(
    name="scomesse/football", 
    api_token = api_key
    )

project[data_version + 'nodes'].download('./nodes.csv.gz')
project[data_version + 'edges'].download('./edges.csv.gz')
project[data_version + 'conversion'].download('./conversion.csv.gz')
project.stop()

https://app.neptune.ai/scomesse/football/
Remember to stop your project once you’ve finished logging your metadata (https://docs.neptune.ai/api/project#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/metadata


### Imports

In [ ]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
print(pd.__version__)
print(np.__version__)

from tqdm import tqdm
from scipy import sparse as sp

1.3.5
1.21.6


In [ ]:
import tensorflow as tf
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.layers import Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [ ]:
from spektral.data.loaders import SingleLoader, DisjointLoader
from spektral.datasets.citation import Citation
from spektral.layers import GCNConv
from spektral.transforms import LayerPreprocess

In [ ]:
from spektral.data import Dataset, Graph
from spektral.transforms.normalize_adj import NormalizeAdj

### Code

In [ ]:
class calcio_graph(Dataset):
    def __init__(self, amount = 500_000, left = 0, train = 350_000, validation = 10_000, **kwargs):
        self.amount = amount
        self.left = left
        self.train = train
        self.validation = validation
        self.mask_tr = self.mask_va = self.mask_te = None
        super().__init__(**kwargs)
    def read(self):
        print('Load nodes...')
        x_df = pd.read_csv('./nodes.csv.gz', usecols = [0, 1, 2], header = 0, names = ['idx','feat', 'y'])
        lower_slice = -self.left
        if self.left == 0:
            lower_slice = None            
        x_df = x_df.iloc[-self.amount - self.left:lower_slice, :].set_index('idx').sort_index()
        x = np.zeros((len(x_df), max(x_df.feat) + 1)).astype(np.float32)
        for cnt, feat in enumerate(x_df.feat):
            x[cnt, feat] = 1
        print('Load edges...')
        start_row = x_df.index.min()
        end_row = x_df.index.max()
        edges_df = pd.read_csv('./edges.csv.gz')
        #edges_df = edges_df[(edges_df.source < self.amount) & (edges_df.target < self.amount)]
        edges_df = edges_df[(edges_df.source >= start_row) & \
                            (edges_df.source <= end_row) & \
                            (edges_df.target >= start_row) & \
                            (edges_df.target <= end_row)]
        edges_df.source = edges_df.source - start_row
        edges_df.target = edges_df.target - start_row
        a = sp.csr_matrix((
            np.ones(len(edges_df)), 
            (edges_df.source.values, edges_df.target.values)
                            ), shape=None).astype(np.float32)
        y = x_df.y.values.reshape(-1,1).astype(np.float32)
        self.mask_tr = np.arange(self.amount) < int(self.train)
        self.mask_va = (np.arange(self.amount) >= int(self.train)) & \
                        (np.arange(self.amount) < int(self.amount - self.validation))
        self.mask_te = np.arange(self.amount) >= int(self.amount - self.validation)
        return [Graph(x = x, a = a, y = y)]

In [ ]:
class SGCN:
    def __init__(self, K):
        self.K = K

    def __call__(self, graph):
        out = graph.a
        for _ in range(self.K - 1):
            out = out.dot(out)
        out.sort_indices()
        graph.a = out
        return graph

In [ ]:
K = 2
dataset = calcio_graph(
                    amount = 500_000,
                    left = 10_000,
                    #transforms=[LayerPreprocess(GCNConv)]#, SGCN(K)]
                    #transforms=[SGCN(K)]
                    #transforms=[LayerPreprocess(GCNConv), AdjToSpTensor()]
                    #transforms=[LayerPreprocess(ARMAConv)]
                    #transforms=[NormalizeAdj()]
                    )

Load nodes...
Load edges...


In [ ]:
dataset[0]

Graph(n_nodes=500000, n_node_features=793, n_edge_features=None, n_labels=1)

In [ ]:
print(dataset[0].a)

In [ ]:
mask_tr, mask_va, mask_te = dataset.mask_tr, dataset.mask_va, dataset.mask_te

In [ ]:
mask_tr.sum(), mask_va.sum(), mask_te.sum()

(350000, 140000, 10000)

In [ ]:
# Parameters
l2_reg = 5e-6  # L2 regularization rate
learning_rate = 0.005  # Learning rate
epochs = 200  # Number of training epochs
patience = 50  # Patience for early stopping
a_dtype = dataset[0].a.dtype  # Only needed for TF 2.1

In [ ]:
N = dataset.n_nodes  # Number of nodes in the graph
F = dataset.n_node_features  # Original size of node features
n_out = dataset.n_labels  # Number of classes

In [ ]:
N, F, n_out

(500000, 793, 1)

In [ ]:
# Model definition
x_in = Input(shape=(F,))
a_in = Input((N,), sparse=True, dtype=a_dtype)

In [ ]:
output = GCNConv(
    n_out, activation = "sigmoid", kernel_regularizer=l2(l2_reg), use_bias=False
)([x_in, a_in])

In [ ]:
# Build model
model = Model(inputs=[x_in, a_in], outputs=output)
optimizer = Adam(lr=learning_rate)
model.compile(
    optimizer=optimizer, loss="binary_crossentropy", weighted_metrics=["acc"]
)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 793)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 500000)]     0           []                               
                                                                                                  
 gcn_conv_1 (GCNConv)           (None, 1)            793         ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
Total params: 793
Trainable params: 793
Non-trainable params: 0
____________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Train model
loader_tr = SingleLoader(dataset, sample_weights=mask_tr)
loader_va = SingleLoader(dataset, sample_weights=mask_va)

In [ ]:
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=1000,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4847 - acc: 0.5190 - val_loss: 0.1936 - val_acc: 0.5421
Epoch 2/1000
1/1 [==============================] - 1s 662ms/step - loss: 0.4839 - acc: 0.5431 - val_loss: 0.1933 - val_acc: 0.5655
Epoch 3/1000
1/1 [==============================] - 1s 656ms/step - loss: 0.4831 - acc: 0.5690 - val_loss: 0.1930 - val_acc: 0.5858
Epoch 4/1000
1/1 [==============================] - 1s 662ms/step - loss: 0.4824 - acc: 0.5901 - val_loss: 0.1927 - val_acc: 0.6025
Epoch 5/1000
1/1 [==============================] - 1s 665ms/step - loss: 0.4816 - acc: 0.6086 - val_loss: 0.1924 - val_acc: 0.6190
Epoch 6/1000
1/1 [==============================] - 1s 652ms/step - loss: 0.4808 - acc: 0.6273 - val_loss: 0.1921 - val_acc: 0.6344
Epoch 7/1000
1/1 [==============================] - 1s 663ms/step - loss: 0.4800 - acc: 0.6430 - val_loss: 0.1918 - val_acc: 0.6469
Epoch 8/1000
1/1 [==============================] - 1s 663ms/step - loss: 0.479

KeyboardInterrupt: ignored

In [ ]:
eval_results = model.evaluate(loader_va.load(), steps=loader_va.steps_per_epoch)
print("Done.\n" "Val loss: {}\n" "Val accuracy: {}".format(*eval_results))

1/1 [==============================] - 2s 2s/step - loss: 0.1418 - acc: 0.7296
Done.
Val loss: 0.14181582629680634
Val accuracy: 0.7296142578125


In [ ]:
# Evaluate model
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=mask_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 1s 1s/step - loss: 0.0293 - acc: 0.5293
Done.
Test loss: 0.029276948422193527
Test accuracy: 0.5292999744415283


In [ ]:
predict_production = model.predict(loader_va.load(), steps=loader_va.steps_per_epoch)

1/1 [==============================] - 1s 1s/step


In [ ]:
nodes_df = pd.read_csv('./nodes.csv.gz', usecols = [0, 2, 6], header = 0, names = ['idx', 'y', 'match_id']).iloc[-509_000:-9_000, :].set_index('idx').sort_index()

In [ ]:
predict_df = pd.DataFrame(columns = ['home', 'not_home'])

In [ ]:
predict_df['home'] = nodes_df[mask_va].y.values[::2]
predict_df['not_home'] = nodes_df[mask_va].y.values[1::2]

In [ ]:
predict_df['pred_home'] = predict_production[mask_va][::2]
predict_df['pred_not_home'] = predict_production[mask_va][1::2]

In [ ]:
predict_df['result'] = (predict_df['pred_home'] > predict_df['pred_not_home']).astype(int)

In [ ]:
(predict_df['home'] == predict_df['result']).sum()

35442

In [ ]:
predict_df['idx_home'] = nodes_df.index[mask_te][::2]
predict_df['idx_not_home'] = nodes_df.index[mask_te][1::2]

In [ ]:
predict_df['match_id_home'] = nodes_df[mask_te].match_id.values[::2]
predict_df['match_id_not_home'] = nodes_df[mask_te].match_id.values[1::2]

In [ ]:
predict_df[predict_df.match_id_home != predict_df.match_id_not_home]

,home,not_home,pred_home,pred_not_home,result,idx_home,idx_not_home,match_id_home,match_id_not_home


In [ ]:
predict_df

,home,not_home,pred_home,pred_not_home,result
0,0,1,0.817898,0.496136,1
1,0,1,0.181242,0.487676,0
2,1,0,0.478412,0.515747,0
3,1,0,0.153708,0.828872,0
4,1,0,0.509116,0.504974,1
...,...,...,...,...,...
69995,1,0,0.314261,0.688905,0
69996,1,0,0.689033,0.312998,1
69997,0,1,0.310236,0.325164,0
69998,1,0,0.326383,0.326463,0


In [ ]:
validation_dataset_name = './prem_validation.csv'
project = neptune.init_project(
    name="scomesse/football", 
    api_token = api_key
    )
project['data/validation_prem_220818'].download(validation_dataset_name)
project.stop()

https://app.neptune.ai/scomesse/football/
Remember to stop your project once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/project#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/metadata


In [ ]:
prem_df = pd.read_csv('./prem_validation.csv')

In [ ]:
prem_df = prem_df[prem_df.Id.isin(list(predict_df.match_id_home))]

In [ ]:
predict_dict = predict_df[predict_df.match_id_home.isin(list(prem_df.Id))].set_index('match_id_home')[['home', 'result']].to_dict(orient = 'index')

In [ ]:
prem_df[['result', 'predict']] = [[predict_dict[key]['home'], predict_dict[key]['result']] for key in prem_df.Id.values]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [ ]:
prem_df

,Id,Idbook,W1,X,W2,W1live,Xlive,W2live,result,predict
7614,2eqn3i74041ragwpuzow7gbo4,13812880,3.600,4.65,1.700,3.600,4.65,1.700,0,0
7615,59i869it1synl0z3di62wss9g,13812877,2.880,4.15,2.030,2.880,4.15,2.030,1,1
7616,dj1uww92e3x26gg2gw79r6g44,13812875,1.116,8.80,14.000,1.116,8.80,14.000,1,0
7618,4hkznpu6g79a8h32ionlfagpg,13813781,1.760,4.10,3.720,1.760,4.10,3.720,1,1
7619,b232tm66r4b7jsi881a6mo6qc,13813779,2.550,3.92,2.300,2.420,3.94,2.410,0,0
...,...,...,...,...,...,...,...,...,...,...
7987,dgtvqq4bwqkgnal8dx3zwccgk,13857861,3.700,3.42,1.930,3.700,3.42,1.930,1,1
7988,1nb3ab0vbihddp9qh2qviix3o,13857924,2.240,4.25,2.510,2.240,4.25,2.510,0,0
7989,9q0423juf175stwpppkxi9kic,13857923,1.250,5.80,9.200,1.250,5.80,9.200,1,1
7990,3y4e7iany0lgbj75cp30g8t90,13857930,8.700,7.10,1.210,8.700,7.10,1.210,0,0


In [ ]:
k1_win_list = []
bet_cnt = 0
bet_win = 0
for k1, res, pred in zip(prem_df.W1live, prem_df.result, prem_df.predict):
    if pred == 1:
        bet_cnt += 1
        if res == 1:
            k1_win_list.append(k1 - 1)
            bet_win += 1
        else:
            k1_win_list.append(-1)
    else:
        k1_win_list.append(0)
print('Sum: ', round(sum(k1_win_list), 2))
print('Bet count:', bet_cnt)
print('Bet wins:', bet_win, ' - bet/win = ', round(bet_win / bet_cnt, 2))

Sum:  164.69
Bet count: 177
Bet wins: 147  - bet/win =  0.83


In [ ]:
k1_win_list = []
bet_cnt = 0
bet_win = 0
for k1, res, pred in zip(prem_df.W1live, prem_df.result, prem_df.predict):
    if pred == 0:
        bet_cnt += 1
        if res == 0:
            k1_win_list.append(0.95 / (k1 - 1))
            bet_win += 1
        else:
            k1_win_list.append(-1)
    else:
        k1_win_list.append(0)
print('Sum: ', round(sum(k1_win_list), 2))
print('Bet count:', bet_cnt)
print('Bet wins:', bet_win, ' - bet/win = ', round(bet_win / bet_cnt, 2))

Sum:  104.81
Bet count: 198
Bet wins: 166  - bet/win =  0.84


In [ ]:
loader_te = SingleLoader(dataset, sample_weights=mask_te)

In [ ]:
for x in loader_te.load():
    print(x)
    break

((<tf.Tensor: shape=(500100, 793), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fe6f118a290>), <tf.Tensor: shape=(500100, 1), dtype=float32, numpy=
array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [0.]], dtype=float32)>, <tf.Tensor: shape=(500100,), dtype=bool, numpy=array([ True,  True,  True, ..., False, False, False])>)
